# CIFAR10 with CNN

training step版

In [1]:
import tensorflow as tf
from common import make_cifar10_dataset
from common import create_sequential_model
import datetime

%load_ext tensorboard

BATCH_SIZE = 128
EPOCHS = 5

!rm -rf ./logs/

In [2]:
train_ds, valid_ds, test_ds = make_cifar10_dataset(BATCH_SIZE)

2021-09-05 03:23:14.921801: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-09-05 03:23:14.936824: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-09-05 03:23:14.938064: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-09-05 03:23:14.940034: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the app

In [3]:
model = create_sequential_model()

In [4]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [5]:
train_loss = tf.keras.metrics.Mean("train_loss", dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy("train_accuracy", dtype=tf.float32)
valid_loss = tf.keras.metrics.Mean("valid_loss", dtype=tf.float32)
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy("valid_accuracy", dtype=tf.float32)

In [6]:
def train_step(model, optimizer, x_train, y_train):
    with tf.GradientTape() as tape:
        prediction = model(x_train, training=True)
        loss = loss_object(y_train, prediction)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    train_accuracy(y_train, prediction)

def valid_step(model, x_valid, y_valid):
    prediction = model(x_valid, training=False)
    loss = loss_object(y_valid, prediction)

    valid_loss(loss)
    valid_accuracy(y_valid, prediction)


In [7]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
valid_log_dir = 'logs/gradient_tape/' + current_time + '/valid'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
valid_summary_writer = tf.summary.create_file_writer(valid_log_dir)

In [8]:
for epoch in range(EPOCHS):
    for x_train, y_train in train_ds:
        train_step(model, optimizer, x_train, y_train)

    with train_summary_writer.as_default():
        tf.summary.scalar("loss", train_loss.result(), step=epoch)
        tf.summary.scalar("accuracy", train_accuracy.result(), step=epoch)

    for x_valid, y_valid in valid_ds:
        valid_step(model, x_valid, y_valid)

    with valid_summary_writer.as_default():
        tf.summary.scalar("loss", valid_loss.result(), step=epoch)
        tf.summary.scalar("accuracy", valid_accuracy.result(), step=epoch)

    print(f"Epoch {epoch}, Loss: {train_loss.result()}, Accuracy: {train_accuracy.result()}")
    print(f"Epoch {epoch}, Valid Loss: {valid_loss.result()}, Valid Accuracy: {valid_accuracy.result()}")

    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()

2021-09-05 03:23:17.835206: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 122880000 exceeds 10% of free system memory.
2021-09-05 03:23:17.889974: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-09-05 03:23:19.442459: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8202


Epoch 0, Loss: 1.57429039478302, Accuracy: 0.42480000853538513
Epoch 0, Valid Loss: 1.2390410900115967, Valid Accuracy: 0.5612999796867371


2021-09-05 03:23:52.450772: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 122880000 exceeds 10% of free system memory.


Epoch 1, Loss: 1.159473180770874, Accuracy: 0.588824987411499
Epoch 1, Valid Loss: 1.0131794214248657, Valid Accuracy: 0.6432999968528748


2021-09-05 03:24:23.088248: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 122880000 exceeds 10% of free system memory.


Epoch 2, Loss: 0.9696264266967773, Accuracy: 0.6556249856948853
Epoch 2, Valid Loss: 0.8653866648674011, Valid Accuracy: 0.6984999775886536
Epoch 3, Loss: 0.8478372097015381, Accuracy: 0.7009750008583069
Epoch 3, Valid Loss: 0.7813783288002014, Valid Accuracy: 0.7297000288963318
Epoch 4, Loss: 0.7555318474769592, Accuracy: 0.7342749834060669
Epoch 4, Valid Loss: 0.8133010864257812, Valid Accuracy: 0.7207000255584717


In [10]:
for x_test, y_test in test_ds:
    valid_step(model, x_test, y_test)


print(f"Test Loss: {valid_loss.result()}, Test Accuracy: {valid_accuracy.result()}")
valid_loss.reset_states()
valid_accuracy.reset_states()

Test Loss: 0.8318063020706177, Test Accuracy: 0.7110000252723694


In [ ]:
%tensorboard --logdir logs/fit